<a href="https://colab.research.google.com/github/joshfpedro/math-328/blob/main/numerical_calculus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import numpy as np

def approx_derivative(f, x, h=1e-5):

    # Backward Difference
    backward_diff = (f(x) - f(x - h)) / h

    # Forward Difference
    forward_diff = (f(x + h) - f(x)) / h

    # Central Difference
    central_diff = (f(x + h) - f(x - h)) / (2 * h)

    return backward_diff, forward_diff, central_diff

# Example
def example_function(x):
    return np.sin(x)

x_value = 2

backward, forward, central = approx_derivative(example_function, x_value, h=1e-10)

print("Backward Difference:", backward)
print("Forward Difference:", forward)
print("Central Difference:", central)
print("Exact Derivative:", np.cos(x_value))

Backward Difference: -0.4161471167662967
Forward Difference: -0.4161471167662967
Central Difference: -0.4161471167662967
Exact Derivative: -0.4161468365471424


In [61]:
def derivative(f, x, h=1e-5, method='central'):
    """Compute numerical derivative of f at x."""

    if method == 'central':
        return (f(x + h) - f(x - h)) / (2 * h)

    elif method == 'forward':
        return (f(x + h) - f(x)) / h

    elif method == 'backward':
        return (f(x) - f(x - h)) / h

    elif method == 'complex':
        return f(x + 1j * h).imag / h

    else:
        raise ValueError("Method must be 'central', 'forward', 'backward', or 'complex'.")

derivative(example_function, x_value, h=1e-16, method='complex')

np.float64(-0.4161468365471424)